In [1]:
import torch
import json
import os
import warnings
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torchquantum as tq
import numpy as np
import math
from torch.optim import Adam
from torchquantum.measurement import expval_joint_analytical

warnings.simplefilter("ignore")

In [2]:
def load_data_setting(setting):
    folder = "../exps"
    json_file = "data_config.json"
    json_path = os.path.join(folder, setting, json_file)
    with open(json_path, "r") as f:
        config = json.load(f)

    permutation_seed = config["permutation_seed"]
    test_size = config["test_size"]
    partition_seed = config["partition_seed"]
    n_class = config["n_class"]
    
    data_tensors = config["data_tensors"]
    loaded_data_tensors = torch.load(data_tensors)
    data_tr = loaded_data_tensors["data_tr"]
    label_tr = loaded_data_tensors["label_tr"]
    data_te = loaded_data_tensors["data_te"]
    label_te = loaded_data_tensors["label_te"]

    return permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te

In [3]:
def create_training_dataloader(data_tr, label_tr, batch_size):
    training_dataset = TensorDataset(data_tr, label_tr)
    training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    return training_dataloader

### Exp1: 10 classes, 8 clients

In [4]:
# Changing the expectations values to concat the clients' outputs
class QNNsubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*2
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            #for j in range(2*d,2*d+1): # (0,2) (2,4)
            for k in range(self.n_wires):
                    #self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                index = k + d * self.n_wires
                self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z"] for i in range(self.n_wires)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [5]:
class QantumServerSubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*4
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            for j in range(2*d,2*d+2): # (0,2) (2,4)
                for k in range(self.n_wires):
                #for k in range(2):
                    self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                    #index = k + j * 2
                    #self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z", "X"] for i in range(5)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [6]:
class QuantumServer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.coeff = coeff
        self.softmax = nn.Softmax(dim=1)
        #self.fc1 = nn.Linear(in_features=16*4, out_features=10)
        self.q = QantumServerSubModel().to(device)
        #self.q = QantumServerSubModel_96()

    def forward(self, clients_outputs):
        # result = sum(clients_outputs)
        # result = result * self.coeff
        # result = self.softmax(result)
        concatenated_result = torch.cat(clients_outputs, dim=1)
        #result = self.fc1(concatenated_result)
        result = self.q(concatenated_result)
        result = self.softmax(result)

        return result

In [7]:
def train(data, label, clients_models, clients_optimizers, server_model, server_optimizer):
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=True)
    server_model.train(mode=True)

    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.zero_grad()
    server_optimizer.zero_grad()

    num_clients = len(clients_models)
    features_per_client = data.shape[1] // num_clients
    #features_per_client = 64
    clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]
    #print(clients_data[0].shape)

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_pred = client_model(clients_data[i])
        # print("Client pred shape:", client_pred.shape)
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    # print("Client 1 outuput shape:", clients_outputs[0].shape)
    result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    # print("Result shape:", result.shape)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_tr, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"train loss: {loss.item():.5f}, train acc: {acc:.3f}", end=' ')
    loss.backward()
    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.step()
    server_optimizer.step()

    return loss.item(), acc


def test(data, label, clients_models, server_model):
    num_clients = len(clients_models)
    features_per_client = data.shape[1] // num_clients
    #features_per_client = 64
    clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=False)
        with torch.no_grad():
            client_pred = client_model(clients_data[i])
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    with torch.no_grad():
        result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    print(f"test loss: {loss.item():.5f} test acc: {acc:.4f}")
    return loss.item(), acc

In [8]:
def run_n_save_exp_8clients(setting_folder, save_path, device):

    # Load data
    permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te = load_data_setting(setting_folder)
    print("PERMUTATION SEED", permutation_seed)
    training_dataloader = create_training_dataloader(data_tr, label_tr, batch_size=32)

    
    # Hyperparameters
    max_epochs = 30
    lr = 0.002

    
    # Models' initialization
    server_model = QuantumServer().to(device)
    server_optimizer = Adam(server_model.parameters(), lr=lr)
    client1_model = QNNsubModel().to(device)
    client1_optimizer = Adam(client1_model.parameters(), lr=lr)
    client2_model = QNNsubModel().to(device)
    client2_optimizer = Adam(client2_model.parameters(), lr=lr)
    client3_model = QNNsubModel().to(device)
    client3_optimizer = Adam(client3_model.parameters(), lr=lr)
    client4_model = QNNsubModel().to(device)
    client4_optimizer = Adam(client4_model.parameters(), lr=lr)
    client5_model = QNNsubModel().to(device)
    client5_optimizer = Adam(client5_model.parameters(), lr=lr)
    client6_model = QNNsubModel().to(device)
    client6_optimizer = Adam(client6_model.parameters(), lr=lr)
    client7_model = QNNsubModel().to(device)
    client7_optimizer = Adam(client7_model.parameters(), lr=lr)
    client8_model = QNNsubModel().to(device)
    client8_optimizer = Adam(client8_model.parameters(), lr=lr)
    
    clients_models = {"client1_model": client1_model,
                     "client2_model": client2_model,
                     "client3_model": client3_model,
                     "client4_model": client4_model,
                     "client5_model": client5_model,
                     "client6_model": client6_model,
                     "client7_model": client7_model,
                     "client8_model": client8_model}
    clients_optimizers = {"client1_optimizer": client1_optimizer,
                         "client2_optimizer": client2_optimizer,
                         "client3_optimizer": client3_optimizer,
                         "client4_optimizer": client4_optimizer,
                         "client5_optimizer": client5_optimizer,
                         "client6_optimizer": client6_optimizer,
                         "client7_optimizer": client7_optimizer,
                         "client8_optimizer": client8_optimizer}

    
    # Trainining and testing
    all_tr_loss = []
    all_test_loss = []
    all_tr_acc = []
    all_test_acc = []
    for i_epoch in range(max_epochs):
        epoch_loss = 0.0 
        epoch_accuracy = 0.0 
        total_samples = 0 
        print(f"Epoch {i_epoch}:", end=" ")
        for batch_X, batch_y in training_dataloader:
            #print(batch_X.shape, batch_y.shape)
            loss_tr, acc_tr = train(batch_X, batch_y, clients_models, clients_optimizers, server_model, server_optimizer)
            batch_size = batch_X.size(0)
            epoch_loss += loss_tr * batch_size  # Scale loss by batch size
            epoch_accuracy += acc_tr * batch_size  # Scale accuracy by batch size
            total_samples += batch_size  # Update sample count
        epoch_loss /= total_samples
        epoch_accuracy /= total_samples
        print(f"Training loss:{epoch_loss:.4f} Training acc:{epoch_accuracy:.4f}")
        loss_test, acc_test = test(data_te, label_te, clients_models, server_model)
        all_tr_loss.append(epoch_loss)
        all_test_loss.append(loss_test)
        all_tr_acc.append(epoch_accuracy)
        all_test_acc.append(acc_test)

    
    # Save models & parameters
    models_tensors_path = save_path+"/"+"models_tensors.pth"
    torch.save({
        "client1_model": clients_models["client1_model"].state_dict(),
        "client2_model": clients_models["client2_model"].state_dict(),
        "client3_model": clients_models["client3_model"].state_dict(),
        "client4_model": clients_models["client4_model"].state_dict(),
        "client5_model": clients_models["client5_model"].state_dict(),
        "client6_model": clients_models["client6_model"].state_dict(),
        "client7_model": clients_models["client7_model"].state_dict(),
        "client8_model": clients_models["client8_model"].state_dict(),
        "server_model": server_model.state_dict(),
    }, models_tensors_path)
    
    params = {
        "permutation_seed": permutation_seed, 
        "test_size": test_size, 
        "partition_seed": partition_seed,
        "n_class": n_class,
        "all_tr_loss": all_tr_loss, 
        "all_test_loss": all_test_loss, 
        "all_tr_acc": all_tr_acc, 
        "all_test_acc": all_test_acc,
        "models_tensors": models_tensors_path
    }
    
    exp_path = save_path+"/"+"exp.json"
    with open(exp_path, "w") as f:
        json.dump(params, f, indent=4)



In [9]:
settings = ["Setting_1", "Setting_2", "Setting_3", "Setting_4", "Setting_5"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_8clients(setting, path, device)

PERMUTATION SEED 42
Epoch 0: train loss: 2.30092, train acc: 0.250 train loss: 2.29691, train acc: 0.125 train loss: 2.29379, train acc: 0.219 train loss: 2.30469, train acc: 0.125 train loss: 2.30025, train acc: 0.156 train loss: 2.29952, train acc: 0.188 train loss: 2.29296, train acc: 0.281 train loss: 2.29116, train acc: 0.344 train loss: 2.28756, train acc: 0.344 train loss: 2.28331, train acc: 0.281 train loss: 2.28272, train acc: 0.344 train loss: 2.27957, train acc: 0.469 train loss: 2.28118, train acc: 0.469 train loss: 2.27951, train acc: 0.500 train loss: 2.27442, train acc: 0.500 train loss: 2.27322, train acc: 0.375 train loss: 2.27335, train acc: 0.250 train loss: 2.26930, train acc: 0.312 train loss: 2.25852, train acc: 0.375 train loss: 2.26169, train acc: 0.562 train loss: 2.26168, train acc: 0.312 train loss: 2.25750, train acc: 0.438 train loss: 2.26109, train acc: 0.594 train loss: 2.26363, train acc: 0.562 train loss: 2.25535, train acc: 0.500 train loss: 2.23912, 

### Exp2: 10 classes, 1 client

In [20]:
# Changing the expectations values to concat the clients' outputs
class QNNsubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*2
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            #for j in range(2*d,2*d+1): # (0,2) (2,4)
            for k in range(self.n_wires):
                    #self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                index = k + d * self.n_wires
                self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z"] for i in range(self.n_wires)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [21]:
class QantumServerSubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        #self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*4
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires)
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            #for j in range(2*d,2*d+2): # (0,2) (2,4)
                #for k in range(self.n_wires):
                #for k in range(2):
                    #self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                    #index = k + j * 2
                    #self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for k in range(self.n_wires):
                self.encoder_gates_x[k](qdev, wires=k, params=x[:, (k)])
                
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z", "X"] for i in range(5)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [22]:
class QuantumServer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.coeff = coeff
        self.softmax = nn.Softmax(dim=1)
        #self.fc1 = nn.Linear(in_features=16*4, out_features=10)
        self.q = QantumServerSubModel().to(device)
        #self.q = QantumServerSubModel_96()

    def forward(self, clients_outputs):
        # result = sum(clients_outputs)
        # result = result * self.coeff
        # result = self.softmax(result)
        concatenated_result = torch.cat(clients_outputs, dim=1)
        #result = self.fc1(concatenated_result)
        result = self.q(concatenated_result)
        result = self.softmax(result)

        return result

In [23]:
def train(data, label, clients_models, clients_optimizers, server_model, server_optimizer):
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=True)
    server_model.train(mode=True)

    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.zero_grad()
    server_optimizer.zero_grad()

    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    features_per_client = 32
    clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]
    #clients_data = [data[:,0:32] for i in range(num_clients)]
    print(clients_data[0].shape)

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_pred = client_model(clients_data[i])
        # print("Client pred shape:", client_pred.shape)
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    # print("Client 1 outuput shape:", clients_outputs[0].shape)
    result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    # print("Result shape:", result.shape)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_tr, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"train loss: {loss.item():.5f}, train acc: {acc:.3f}", end=' ')
    loss.backward()
    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.step()
    server_optimizer.step()

    return loss.item(), acc


def test(data, label, clients_models, server_model):
    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    features_per_client = 32
    clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]
    #clients_data = [data[:,0:32] for i in range(num_clients)]

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=False)
        with torch.no_grad():
            client_pred = client_model(clients_data[i])
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    with torch.no_grad():
        result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    print(f"test loss: {loss.item():.5f} test acc: {acc:.3f}")
    return loss.item(), acc

In [24]:
def run_n_save_exp_1client(setting_folder, save_path, device):

    # Load data
    permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te = load_data_setting(setting_folder)
    print("PERMUTATION SEED", permutation_seed)
    training_dataloader = create_training_dataloader(data_tr, label_tr, batch_size=32)

    
    # Hyperparameters
    max_epochs = 30
    lr = 0.002

    
    # Models' initialization
    server_model = QuantumServer().to(device)
    server_optimizer = Adam(server_model.parameters(), lr=lr)
    client1_model = QNNsubModel().to(device)
    client1_optimizer = Adam(client1_model.parameters(), lr=lr)
    
    clients_models = {"client1_model": client1_model}
    clients_optimizers = {"client1_optimizer": client1_optimizer}

    
    # Trainining and testing
    all_tr_loss = []
    all_test_loss = []
    all_tr_acc = []
    all_test_acc = []
    for i_epoch in range(max_epochs):
        epoch_loss = 0.0 
        epoch_accuracy = 0.0 
        total_samples = 0 
        print(f"Epoch {i_epoch}:", end=" ")
        for batch_X, batch_y in training_dataloader:
            print(batch_X.shape, batch_y.shape)
            loss_tr, acc_tr = train(batch_X, batch_y, clients_models, clients_optimizers, server_model, server_optimizer)
            batch_size = batch_X.size(0)
            epoch_loss += loss_tr * batch_size  # Scale loss by batch size
            epoch_accuracy += acc_tr * batch_size  # Scale accuracy by batch size
            total_samples += batch_size  # Update sample count
        epoch_loss /= total_samples
        epoch_accuracy /= total_samples
        print(f"Training loss:{epoch_loss:.4f} Training acc:{epoch_accuracy:.4f}")
        loss_test, acc_test = test(data_te, label_te, clients_models, server_model)
        all_tr_loss.append(epoch_loss)
        all_test_loss.append(loss_test)
        all_tr_acc.append(epoch_accuracy)
        all_test_acc.append(acc_test)

    
    # Save models & parameters
    models_tensors_path = save_path+"/"+"models_tensors_1.pth"
    torch.save({
        "client1_model": clients_models["client1_model"].state_dict(),
        "server_model": server_model.state_dict(),
    }, models_tensors_path)
    
    params = {
        "permutation_seed": permutation_seed, 
        "test_size": test_size, 
        "partition_seed": partition_seed,
        "n_class": n_class,
        "all_tr_loss": all_tr_loss, 
        "all_test_loss": all_test_loss, 
        "all_tr_acc": all_tr_acc, 
        "all_test_acc": all_test_acc,
        "models_tensors": models_tensors_path
    }
    
    exp_path = save_path+"/"+"exp1.json"
    with open(exp_path, "w") as f:
        json.dump(params, f, indent=4)



In [25]:
settings = ["Setting_1", "Setting_2", "Setting_3"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_1client(setting, path, device)

PERMUTATION SEED 42
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30190, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29982, train acc: 0.031 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30663, train acc: 0.000 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30366, train acc: 0.000 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29635, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.31179, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.31207, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29954, train acc: 0.156 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29147, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29450, train ac

In [26]:
settings = ["Setting_4", "Setting_5"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_1client(setting, path, device)

PERMUTATION SEED 45
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30212, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30581, train acc: 0.031 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29815, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30169, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30625, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30638, train acc: 0.031 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29968, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30248, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30179, train acc: 0.156 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30921, train ac

### Exps3: 10 classes, 8x1 client

In [4]:
# Changing the expectations values to concat the clients' outputs
class QNNsubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*2
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            #for j in range(2*d,2*d+1): # (0,2) (2,4)
            for k in range(self.n_wires):
                    #self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                index = k + d * self.n_wires
                self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z"] for i in range(self.n_wires)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [5]:
class QantumServerSubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*4
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            for j in range(2*d,2*d+2): # (0,2) (2,4)
                for k in range(self.n_wires):
                #for k in range(2):
                    self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                    #index = k + j * 2
                    #self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z", "X"] for i in range(5)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [6]:
class QuantumServer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.coeff = coeff
        self.softmax = nn.Softmax(dim=1)
        #self.fc1 = nn.Linear(in_features=16*4, out_features=10)
        self.q = QantumServerSubModel().to(device)
        #self.q = QantumServerSubModel_96()

    def forward(self, clients_outputs):
        # result = sum(clients_outputs)
        # result = result * self.coeff
        # result = self.softmax(result)
        concatenated_result = torch.cat(clients_outputs, dim=1)
        #result = self.fc1(concatenated_result)
        result = self.q(concatenated_result)
        result = self.softmax(result)

        return result

In [7]:
def train(data, label, clients_models, clients_optimizers, server_model, server_optimizer):
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=True)
    server_model.train(mode=True)

    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.zero_grad()
    server_optimizer.zero_grad()

    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    ##features_per_client = 64
    #clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]
    clients_data = [data[:,0:32] for i in range(num_clients)]
    print(clients_data[0].shape)

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_pred = client_model(clients_data[i])
        # print("Client pred shape:", client_pred.shape)
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    # print("Client 1 outuput shape:", clients_outputs[0].shape)
    result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    # print("Result shape:", result.shape)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_tr, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"train loss: {loss.item():.5f}, train acc: {acc:.3f}", end=' ')
    loss.backward()
    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.step()
    server_optimizer.step()

    return loss.item(), acc


def test(data, label, clients_models, server_model):
    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    ##features_per_client = 64
    #clients_data = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(num_clients)]
    clients_data = [data[:,0:32] for i in range(num_clients)]

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=False)
        with torch.no_grad():
            client_pred = client_model(clients_data[i])
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    with torch.no_grad():
        result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    print(f"test loss: {loss.item():.5f} test acc: {acc:.3f}")
    return loss.item(), acc

In [8]:
def run_n_save_exp_8x1client(setting_folder, save_path, device):

    # Load data
    permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te = load_data_setting(setting_folder)
    print("PERMUTATION SEED", permutation_seed)
    training_dataloader = create_training_dataloader(data_tr, label_tr, batch_size=32)

    
    # Hyperparameters
    max_epochs = 30
    lr = 0.002

    
    # Models' initialization
    server_model = QuantumServer().to(device)
    server_optimizer = Adam(server_model.parameters(), lr=lr)
    client1_model = QNNsubModel().to(device)
    client1_optimizer = Adam(client1_model.parameters(), lr=lr)
    client2_model = QNNsubModel().to(device)
    client2_optimizer = Adam(client2_model.parameters(), lr=lr)
    client3_model = QNNsubModel().to(device)
    client3_optimizer = Adam(client3_model.parameters(), lr=lr)
    client4_model = QNNsubModel().to(device)
    client4_optimizer = Adam(client4_model.parameters(), lr=lr)
    client5_model = QNNsubModel().to(device)
    client5_optimizer = Adam(client5_model.parameters(), lr=lr)
    client6_model = QNNsubModel().to(device)
    client6_optimizer = Adam(client6_model.parameters(), lr=lr)
    client7_model = QNNsubModel().to(device)
    client7_optimizer = Adam(client7_model.parameters(), lr=lr)
    client8_model = QNNsubModel().to(device)
    client8_optimizer = Adam(client8_model.parameters(), lr=lr)
    
    clients_models = {"client1_model": client1_model,
                     "client2_model": client2_model,
                     "client3_model": client3_model,
                     "client4_model": client4_model,
                     "client5_model": client5_model,
                     "client6_model": client6_model,
                     "client7_model": client7_model,
                     "client8_model": client8_model}
    clients_optimizers = {"client1_optimizer": client1_optimizer,
                         "client2_optimizer": client2_optimizer,
                         "client3_optimizer": client3_optimizer,
                         "client4_optimizer": client4_optimizer,
                         "client5_optimizer": client5_optimizer,
                         "client6_optimizer": client6_optimizer,
                         "client7_optimizer": client7_optimizer,
                         "client8_optimizer": client8_optimizer}

    
    # Trainining and testing
    all_tr_loss = []
    all_test_loss = []
    all_tr_acc = []
    all_test_acc = []
    for i_epoch in range(max_epochs):
        epoch_loss = 0.0 
        epoch_accuracy = 0.0 
        total_samples = 0 
        print(f"Epoch {i_epoch}:", end=" ")
        for batch_X, batch_y in training_dataloader:
            print(batch_X.shape, batch_y.shape)
            loss_tr, acc_tr = train(batch_X, batch_y, clients_models, clients_optimizers, server_model, server_optimizer)
            batch_size = batch_X.size(0)
            epoch_loss += loss_tr * batch_size  # Scale loss by batch size
            epoch_accuracy += acc_tr * batch_size  # Scale accuracy by batch size
            total_samples += batch_size  # Update sample count
        epoch_loss /= total_samples
        epoch_accuracy /= total_samples
        print(f"Training loss:{epoch_loss:.4f} Training acc:{epoch_accuracy:.4f}")
        loss_test, acc_test = test(data_te, label_te, clients_models, server_model)
        all_tr_loss.append(epoch_loss)
        all_test_loss.append(loss_test)
        all_tr_acc.append(epoch_accuracy)
        all_test_acc.append(acc_test)

    
    # Save models & parameters
    models_tensors_path = save_path+"/"+"models_tensors_8x1.pth"
    torch.save({
        "client1_model": clients_models["client1_model"].state_dict(),
        "client2_model": clients_models["client2_model"].state_dict(),
        "client3_model": clients_models["client3_model"].state_dict(),
        "client4_model": clients_models["client4_model"].state_dict(),
        "client5_model": clients_models["client5_model"].state_dict(),
        "client6_model": clients_models["client6_model"].state_dict(),
        "client7_model": clients_models["client7_model"].state_dict(),
        "client8_model": clients_models["client8_model"].state_dict(),
        "server_model": server_model.state_dict(),
    }, models_tensors_path)
    
    params = {
        "permutation_seed": permutation_seed, 
        "test_size": test_size, 
        "partition_seed": partition_seed,
        "n_class": n_class,
        "all_tr_loss": all_tr_loss, 
        "all_test_loss": all_test_loss, 
        "all_tr_acc": all_tr_acc, 
        "all_test_acc": all_test_acc,
        "models_tensors": models_tensors_path
    }
    
    exp_path = save_path+"/"+"exp8x1.json"
    with open(exp_path, "w") as f:
        json.dump(params, f, indent=4)



In [ ]:
settings = ["Setting_2"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_8x1client(setting, path, device)

PERMUTATION SEED 43
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30333, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30093, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.31154, train acc: 0.000 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29612, train acc: 0.156 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29994, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29584, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28007, train acc: 0.250 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29704, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29071, train acc: 0.156 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28334, train ac

In [9]:
settings = ["Setting_3"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_8x1client(setting, path, device)

PERMUTATION SEED 44
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30307, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30016, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30156, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30229, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29547, train acc: 0.250 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28982, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28988, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29216, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28576, train acc: 0.281 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.27869, train ac

In [10]:
settings = ["Setting_4"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_8x1client(setting, path, device)

PERMUTATION SEED 45
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30387, train acc: 0.156 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30269, train acc: 0.094 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30283, train acc: 0.250 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29926, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30424, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29215, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30194, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29194, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29544, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28375, train ac

In [9]:
settings = ["Setting_5"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_8x1client(setting, path, device)

PERMUTATION SEED 46
Epoch 0: torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30086, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.30811, train acc: 0.062 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29242, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28820, train acc: 0.188 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29884, train acc: 0.031 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29140, train acc: 0.125 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28703, train acc: 0.250 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.28914, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29035, train acc: 0.219 torch.Size([32, 256]) torch.Size([32])
torch.Size([32, 32])
train loss: 2.29158, train ac

### Exp4: 10 classes, 2x4 clients

In [11]:
# Changing the expectations values to concat the clients' outputs
class QNNsubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*2
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            #for j in range(2*d,2*d+1): # (0,2) (2,4)
            for k in range(self.n_wires):
                    #self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                index = k + d * self.n_wires
                self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z"] for i in range(self.n_wires)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [12]:
class QantumServerSubModel(nn.Module):
    def __init__(self, n_qubits=8, n_block=5, n_depth_per_block=100):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*4
        self.n_block = n_block
        self.n_depth_per_block = n_depth_per_block
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            for j in range(2*d,2*d+2): # (0,2) (2,4)
                for k in range(self.n_wires):
                #for k in range(2):
                    self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
                    #index = k + j * 2
                    #self.encoder_gates_x[index](qdev, wires=k, params=x[:, (index)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["Z", "X"] for i in range(5)]
        ret = torch.stack(obs_list, dim=1)
        return ret

In [13]:
class QuantumServer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.coeff = coeff
        self.softmax = nn.Softmax(dim=1)
        #self.fc1 = nn.Linear(in_features=16*4, out_features=10)
        self.q = QantumServerSubModel().to(device)
        #self.q = QantumServerSubModel_96()

    def forward(self, clients_outputs):
        # result = sum(clients_outputs)
        # result = result * self.coeff
        # result = self.softmax(result)
        concatenated_result = torch.cat(clients_outputs, dim=1)
        #result = self.fc1(concatenated_result)
        result = self.q(concatenated_result)
        result = self.softmax(result)

        return result

In [14]:
def train(data, label, clients_models, clients_optimizers, server_model, server_optimizer):
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=True)
    server_model.train(mode=True)

    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.zero_grad()
    server_optimizer.zero_grad()

    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    features_per_client = 32
    clients_data1 = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(4)]
    clients_data = np.concatenate((clients_data1, clients_data1))
    print(clients_data[0].shape)

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_pred = client_model(clients_data[i])
        # print("Client pred shape:", client_pred.shape)
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    # print("Client 1 outuput shape:", clients_outputs[0].shape)
    result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    # print("Result shape:", result.shape)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_tr, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"train loss: {loss.item():.5f}, train acc: {acc:.3f}", end=' ')
    loss.backward()
    for key, client_optimizer in clients_optimizers.items():
        client_optimizer.step()
    server_optimizer.step()

    return loss.item(), acc


def test(data, label, clients_models, server_model):
    num_clients = len(clients_models)
    #features_per_client = data.shape[1] // num_clients
    features_per_client = 32
    clients_data1 = [data[:, i * features_per_client:(i + 1) * features_per_client] for i in range(4)]
    clients_data = np.concatenate((clients_data1, clients_data1))

    clients_outputs = []
    for i, client_model in enumerate(clients_models.values()):
        client_model.train(mode=False)
        with torch.no_grad():
            client_pred = client_model(clients_data[i])
        clients_outputs.append(client_pred)

    #result = sum(clients_outputs)
    with torch.no_grad():
        result = server_model(clients_outputs)
    # result = clients_outputs[0] + clients_outputs[1] + clients_outputs[2] + clients_outputs[3]
    # result = server_model(result)
    loss = torch.nn.CrossEntropyLoss()(result, label)
    acc = (result.argmax(axis=1) == label).sum().item() / len(label)
    print(f"test loss: {loss.item():.5f} test acc: {acc:.4f}")
    return loss.item(), acc

In [15]:
def run_n_save_exp_2x4clients(setting_folder, save_path, device):

    # Load data
    permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te = load_data_setting(setting_folder)
    print("PERMUTATION SEED", permutation_seed)
    training_dataloader = create_training_dataloader(data_tr, label_tr, batch_size=32)

    
    # Hyperparameters
    max_epochs = 30
    lr = 0.002

    
    # Models' initialization
    server_model = QuantumServer().to(device)
    server_optimizer = Adam(server_model.parameters(), lr=lr)
    client1_model = QNNsubModel().to(device)
    client1_optimizer = Adam(client1_model.parameters(), lr=lr)
    client2_model = QNNsubModel().to(device)
    client2_optimizer = Adam(client2_model.parameters(), lr=lr)
    client3_model = QNNsubModel().to(device)
    client3_optimizer = Adam(client3_model.parameters(), lr=lr)
    client4_model = QNNsubModel().to(device)
    client4_optimizer = Adam(client4_model.parameters(), lr=lr)
    client5_model = QNNsubModel().to(device)
    client5_optimizer = Adam(client5_model.parameters(), lr=lr)
    client6_model = QNNsubModel().to(device)
    client6_optimizer = Adam(client6_model.parameters(), lr=lr)
    client7_model = QNNsubModel().to(device)
    client7_optimizer = Adam(client7_model.parameters(), lr=lr)
    client8_model = QNNsubModel().to(device)
    client8_optimizer = Adam(client8_model.parameters(), lr=lr)
    
    clients_models = {"client1_model": client1_model,
                     "client2_model": client2_model,
                     "client3_model": client3_model,
                     "client4_model": client4_model,
                     "client5_model": client5_model,
                     "client6_model": client6_model,
                     "client7_model": client7_model,
                     "client8_model": client8_model}
    clients_optimizers = {"client1_optimizer": client1_optimizer,
                         "client2_optimizer": client2_optimizer,
                         "client3_optimizer": client3_optimizer,
                         "client4_optimizer": client4_optimizer,
                         "client5_optimizer": client5_optimizer,
                         "client6_optimizer": client6_optimizer,
                         "client7_optimizer": client7_optimizer,
                         "client8_optimizer": client8_optimizer}

    
    # Trainining and testing
    all_tr_loss = []
    all_test_loss = []
    all_tr_acc = []
    all_test_acc = []
    for i_epoch in range(max_epochs):
        epoch_loss = 0.0 
        epoch_accuracy = 0.0 
        total_samples = 0 
        print(f"Epoch {i_epoch}:", end=" ")
        for batch_X, batch_y in training_dataloader:
            #print(batch_X.shape, batch_y.shape)
            loss_tr, acc_tr = train(batch_X, batch_y, clients_models, clients_optimizers, server_model, server_optimizer)
            batch_size = batch_X.size(0)
            epoch_loss += loss_tr * batch_size  # Scale loss by batch size
            epoch_accuracy += acc_tr * batch_size  # Scale accuracy by batch size
            total_samples += batch_size  # Update sample count
        epoch_loss /= total_samples
        epoch_accuracy /= total_samples
        print(f"Training loss:{epoch_loss:.4f} Training acc:{epoch_accuracy:.4f}")
        loss_test, acc_test = test(data_te, label_te, clients_models, server_model)
        all_tr_loss.append(epoch_loss)
        all_test_loss.append(loss_test)
        all_tr_acc.append(epoch_accuracy)
        all_test_acc.append(acc_test)

    
    # Save models & parameters
    models_tensors_path = save_path+"/"+"models_tensors_2x4.pth"
    torch.save({
        "client1_model": clients_models["client1_model"].state_dict(),
        "client2_model": clients_models["client2_model"].state_dict(),
        "client3_model": clients_models["client3_model"].state_dict(),
        "client4_model": clients_models["client4_model"].state_dict(),
        "client5_model": clients_models["client5_model"].state_dict(),
        "client6_model": clients_models["client6_model"].state_dict(),
        "client7_model": clients_models["client7_model"].state_dict(),
        "client8_model": clients_models["client8_model"].state_dict(),
        "server_model": server_model.state_dict(),
    }, models_tensors_path)
    
    params = {
        "permutation_seed": permutation_seed, 
        "test_size": test_size, 
        "partition_seed": partition_seed,
        "n_class": n_class,
        "all_tr_loss": all_tr_loss, 
        "all_test_loss": all_test_loss, 
        "all_tr_acc": all_tr_acc, 
        "all_test_acc": all_test_acc,
        "models_tensors": models_tensors_path
    }
    
    exp_path = save_path+"/"+"exp2x4.json"
    with open(exp_path, "w") as f:
        json.dump(params, f, indent=4)



In [16]:
settings = ["Setting_1"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_2x4clients(setting, path, device)

PERMUTATION SEED 42
Epoch 0: (32, 32)
train loss: 2.30339, train acc: 0.031 (32, 32)
train loss: 2.30020, train acc: 0.062 (32, 32)
train loss: 2.29232, train acc: 0.250 (32, 32)
train loss: 2.30296, train acc: 0.031 (32, 32)
train loss: 2.29319, train acc: 0.188 (32, 32)
train loss: 2.29760, train acc: 0.156 (32, 32)
train loss: 2.29500, train acc: 0.188 (32, 32)
train loss: 2.29837, train acc: 0.156 (32, 32)
train loss: 2.29369, train acc: 0.125 (32, 32)
train loss: 2.28038, train acc: 0.250 (32, 32)
train loss: 2.28619, train acc: 0.250 (32, 32)
train loss: 2.28279, train acc: 0.344 (32, 32)
train loss: 2.27962, train acc: 0.438 (32, 32)
train loss: 2.28659, train acc: 0.281 (32, 32)
train loss: 2.28024, train acc: 0.438 (32, 32)
train loss: 2.28010, train acc: 0.406 (32, 32)
train loss: 2.27715, train acc: 0.438 (32, 32)
train loss: 2.27709, train acc: 0.531 (32, 32)
train loss: 2.27304, train acc: 0.500 (32, 32)
train loss: 2.27470, train acc: 0.438 (32, 32)
train loss: 2.26766, t

In [17]:
settings = ["Setting_2"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_2x4clients(setting, path, device)

PERMUTATION SEED 43
Epoch 0: (32, 32)
train loss: 2.30433, train acc: 0.062 (32, 32)
train loss: 2.29875, train acc: 0.219 (32, 32)
train loss: 2.29911, train acc: 0.219 (32, 32)
train loss: 2.29217, train acc: 0.094 (32, 32)
train loss: 2.29707, train acc: 0.219 (32, 32)
train loss: 2.28538, train acc: 0.312 (32, 32)
train loss: 2.29279, train acc: 0.312 (32, 32)
train loss: 2.28494, train acc: 0.312 (32, 32)
train loss: 2.28793, train acc: 0.375 (32, 32)
train loss: 2.27999, train acc: 0.375 (32, 32)
train loss: 2.27794, train acc: 0.406 (32, 32)
train loss: 2.27971, train acc: 0.375 (32, 32)
train loss: 2.27631, train acc: 0.438 (32, 32)
train loss: 2.27560, train acc: 0.344 (32, 32)
train loss: 2.27193, train acc: 0.438 (32, 32)
train loss: 2.27166, train acc: 0.375 (32, 32)
train loss: 2.26562, train acc: 0.344 (32, 32)
train loss: 2.28172, train acc: 0.250 (32, 32)
train loss: 2.27156, train acc: 0.375 (32, 32)
train loss: 2.25623, train acc: 0.469 (32, 32)
train loss: 2.25885, t

In [18]:
settings = ["Setting_3"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_2x4clients(setting, path, device)

PERMUTATION SEED 44
Epoch 0: (32, 32)
train loss: 2.30260, train acc: 0.062 (32, 32)
train loss: 2.30624, train acc: 0.094 (32, 32)
train loss: 2.30210, train acc: 0.062 (32, 32)
train loss: 2.30267, train acc: 0.188 (32, 32)
train loss: 2.29724, train acc: 0.188 (32, 32)
train loss: 2.29443, train acc: 0.188 (32, 32)
train loss: 2.28972, train acc: 0.250 (32, 32)
train loss: 2.29024, train acc: 0.312 (32, 32)
train loss: 2.29120, train acc: 0.312 (32, 32)
train loss: 2.28849, train acc: 0.219 (32, 32)
train loss: 2.28101, train acc: 0.406 (32, 32)
train loss: 2.28286, train acc: 0.375 (32, 32)
train loss: 2.28095, train acc: 0.438 (32, 32)
train loss: 2.27862, train acc: 0.375 (32, 32)
train loss: 2.27219, train acc: 0.625 (32, 32)
train loss: 2.27760, train acc: 0.469 (32, 32)
train loss: 2.27331, train acc: 0.469 (32, 32)
train loss: 2.26566, train acc: 0.594 (32, 32)
train loss: 2.26167, train acc: 0.750 (32, 32)
train loss: 2.25647, train acc: 0.781 (32, 32)
train loss: 2.25841, t

In [19]:
settings = ["Setting_4", "Setting_5"]
root = "../exps"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for setting in settings:
    path = os.path.join(root, setting)
    os.makedirs(path, exist_ok=True)
    run_n_save_exp_2x4clients(setting, path, device)

PERMUTATION SEED 45
Epoch 0: (32, 32)
train loss: 2.30245, train acc: 0.188 (32, 32)
train loss: 2.30090, train acc: 0.156 (32, 32)
train loss: 2.29510, train acc: 0.156 (32, 32)
train loss: 2.29670, train acc: 0.250 (32, 32)
train loss: 2.30934, train acc: 0.062 (32, 32)
train loss: 2.29255, train acc: 0.156 (32, 32)
train loss: 2.29242, train acc: 0.125 (32, 32)
train loss: 2.28234, train acc: 0.344 (32, 32)
train loss: 2.28543, train acc: 0.406 (32, 32)
train loss: 2.27309, train acc: 0.531 (32, 32)
train loss: 2.27457, train acc: 0.562 (32, 32)
train loss: 2.26428, train acc: 0.594 (32, 32)
train loss: 2.28694, train acc: 0.219 (32, 32)
train loss: 2.27728, train acc: 0.438 (32, 32)
train loss: 2.28232, train acc: 0.375 (32, 32)
train loss: 2.26482, train acc: 0.531 (32, 32)
train loss: 2.26280, train acc: 0.531 (32, 32)
train loss: 2.27364, train acc: 0.344 (32, 32)
train loss: 2.27357, train acc: 0.219 (32, 32)
train loss: 2.27862, train acc: 0.281 (32, 32)
train loss: 2.27049, t